In [9]:
import torch
import torchvision
import torchvision.transforms as transforms
import idx2numpy
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import PIL
import os
import random
import optuna
import joblib

In [10]:
trainimages = []
trainlabels = []
index = 0
mode = "Train"
Map= {}
emotions = os.listdir(f'emotionfolder/{mode}')
for emotion in emotions:
    Map[index]=emotion

    if emotion=="affectnet" or emotion=="emotionfolder":
        continue
    
    files =os.listdir(f'emotionfolder/{mode}/{emotion}')
    for file in files:
        try:
            filepath =f'emotionfolder/{mode}/{emotion}/{file}'
            trainimages.append(filepath)
            trainlabels.append(index)
        except Exception as e:
            pass
    index+=1
    

In [11]:
testimages = []
testlabels = []
index = 0
mode = 'Test'
emotions = os.listdir(f'emotionfolder/{mode}')
for emotion in emotions:
    if emotion=="affectnet" or emotion=="emotionfolder":
        continue
    
    files =os.listdir(f'emotionfolder/{mode}/{emotion}')
    for file in files:
        try:
            img = f'emotionfolder/{mode}/{emotion}/{file}'
            testimages.append(img)
            testlabels.append(index)
        except Exception as e:
            pass
    index+=1
    

In [12]:
#split is rougly 60/40 right now, need to make it ~80/10/10
trainimages.extend(testimages)
trainlabels.extend(testlabels)

trainimages_final = []
trainlabel_final = []
testimages_final = []
testlabel_final = []
valimages_final = []
vallabels_final = []

In [13]:

for i in range(0, len(trainimages)-1):
    num = random.randint(1,10)

    if 1<=num<=8:
        trainimages_final.append(trainimages[i])
        trainlabel_final.append(trainlabels[i])

    elif num==9:
        testimages_final.append(trainimages[i])
        testlabel_final.append(trainlabels[i])

    else:
        valimages_final.append(trainimages[i])
        vallabels_final.append(trainlabels[i])
        
    

In [14]:
target_size = (96,96)
mean = [0.5,0.5,0.5]
std = [0.5,0.5,0.5]

In [15]:
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(target_size),
    transforms.RandomVerticalFlip(10),
    transforms.RandomHorizontalFlip(9),
    transforms.RandomRotation(12),
    transforms.ColorJitter(brightness=0.3, contrast=0.24),  
    transforms.ToTensor(),
    transforms.Normalize(mean, std)

])
transforms_test = transforms.Compose([
    transforms.Resize(target_size),
    transforms.CenterCrop(target_size),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)])

In [16]:
class MyDataset(Dataset):
    def __init__(self, imageslink, label,transformation_maker):
        super().__init__()
        self.imageslink = imageslink
        self.label = label
        self.transformation_maker = transformation_maker

    def __len__(self):
        return len(self.label)

    def __getitem__(self,idx):
        img = Image.open(self.imageslink[idx])
        img =  self.transformation_maker(img)

        return img, self.label[idx]

In [17]:
trainDataset = MyDataset(trainimages_final, trainlabel_final, transform_train)
testDataset = MyDataset(testimages_final, testlabel_final, transforms_test)
valDataset = MyDataset(valimages_final, vallabels_final, transforms_test)
trainloader = DataLoader(trainDataset, batch_size=256, shuffle=True, num_workers=10, pin_memory=True)
testloader = DataLoader(testDataset, batch_size=256, shuffle=True, num_workers=10, pin_memory=True)
valloader = DataLoader(valDataset, batch_size=256, shuffle=True, num_workers=10, pin_memory=True)
#8outputs

In [18]:
class CnnArchitecture(nn.Module):
    def __init__(self, is_grayscale, output_size, num_layers, hidden_size,  kernel_size, image_size=96):
        super().__init__()

        self.input_size=1 if is_grayscale else 3
        self.output_size = output_size
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.imagesize = image_size
        self.kernelsize = kernel_size
        self.map = {3:1, 5:2, 7:3}
        self.Network = []

        for i in range(num_layers):
            if i==0:
                self.Network.append(nn.Conv2d(self.input_size, self.hidden_size, kernel_size=self.kernelsize, padding=self.map[self.kernelsize]))
                self.Network.append(nn.ReLU())
                self.Network.append(nn.MaxPool2d(kernel_size=2, stride=2))

            else:
                self.Network.append(nn.Conv2d(self.hidden_size, self.hidden_size, kernel_size=self.kernelsize, padding=self.map[self.kernelsize]))
                self.Network.append(nn.ReLU())
                self.Network.append(nn.MaxPool2d(kernel_size=2, stride=2))

        self.features = nn.Sequential(*self.Network)
        sampletorch = torch.randn(1, self.input_size, self.imagesize, self.imagesize)
        self.dummy_size = self.features(sampletorch).numel()

        self.classifier = nn.Linear(self.dummy_size, self.output_size)

    def forward(self,x):
        x=self.features(x)
        x = torch.flatten(x, 1)
        return self.classifier(x)#no relu here since criterion and loss will take care of it!
                                                                              

                                                                              
                                                            
                                                
        

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [20]:
def objective(trial):
    batch_size = trial.suggest_categorical("batch_size", [32,64,128,256])
    num_layers = trial.suggest_categorical("num_layers", [2,4,6])
    hidden_size = trial.suggest_categorical("hidden_size", [16, 64, 128])
    lr = trial.suggest_categorical("lr", [1e-3, 1e-4, 1e-5])
    epochs = trial.suggest_categorical("epochs", [50, 75, 100])
    weight_decay = trial.suggest_categorical("weight_decay", [1e-2, 1e-3, 1e-4])   
    kernel_size = trial.suggest_categorical("kernel_size", [3,5,7])

    

    
    trainloader = DataLoader(trainDataset, batch_size=batch_size, shuffle=True, num_workers=16, pin_memory=True)
    valloader = DataLoader(valDataset, batch_size=batch_size, shuffle=True, num_workers=16, pin_memory=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = CnnArchitecture(False, 8, num_layers, hidden_size, kernel_size)
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    testloss = []

    
    for i in range(epochs):
        model.train()
        training_loss = 0
        for x,y in trainloader:
            x = x.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            y_pred = model(x)
            loss = criterion(y_pred, y)
            training_loss+=loss.item()
            loss.backward()
            optimizer.step()
    
    
        model.eval()
        test_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for x,y in valloader:
                x = x.to(device)
                y = y.to(device)
                y_pred = model(x)
                loss = criterion(y_pred, y)
                test_loss+=loss.item()
                total += y.size(0)
                probability, predicted = torch.max(y_pred.data, 1)
                correct += (predicted == y).sum().item()
                
        if i%20==0 or i==epochs-1: 
            avg_train_loss = training_loss / len(trainloader)
            avg_test_loss = test_loss / len(valloader)
            accuracy = (100 * correct) / total
    
            print(f"Epoch num: {i}")
            print(f"  Train Loss: {avg_train_loss}")
            print(f"  Test Loss:  {avg_test_loss}")
            print(f"  Accuracy:   {accuracy}%")
            print("--------------------")
            testloss.append(avg_test_loss)
            trial.report(avg_test_loss, i)
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

    return testloss[-1]#want to minimize testloss!


In [210]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=40)

[I 2025-11-10 23:25:17,595] A new study created in memory with name: no-name-4e3207bb-491a-4e3e-a1bc-d3662b17d14f


Epoch num: 0
  Train Loss: 2.0499885339910784
  Test Loss:  2.042656548321247
  Accuracy:   17.583848909150113%
--------------------
Epoch num: 20
  Train Loss: 1.8262295167272289
  Test Loss:  2.2791463981072106
  Accuracy:   18.788668186258548%
--------------------
Epoch num: 40
  Train Loss: 1.7114592927197616
  Test Loss:  2.1040219888091087
  Accuracy:   22.27287528492348%
--------------------
Epoch num: 60
  Train Loss: 1.6306624794378877
  Test Loss:  2.192557486395041
  Accuracy:   19.798111364376425%
--------------------
Epoch num: 80
  Train Loss: 1.5445666424930096
  Test Loss:  2.3895897765954337
  Accuracy:   18.528166720937804%
--------------------


[I 2025-11-10 23:35:01,579] Trial 0 finished with value: 2.4875972668329873 and parameters: {'batch_size': 64, 'num_layers': 6, 'hidden_size': 16, 'lr': 0.0001, 'epochs': 100, 'weight_decay': 0.001, 'kernel_size': 7}. Best is trial 0 with value: 2.4875972668329873.


Epoch num: 99
  Train Loss: 1.4856473269561927
  Test Loss:  2.4875972668329873
  Accuracy:   17.42103549332465%
--------------------
Epoch num: 0
  Train Loss: 2.058557524345815
  Test Loss:  2.041880595187346
  Accuracy:   17.583848909150113%
--------------------
Epoch num: 20
  Train Loss: 1.8383483868092299
  Test Loss:  2.283037394285202
  Accuracy:   17.42103549332465%
--------------------
Epoch num: 40
  Train Loss: 1.7198353726416826
  Test Loss:  2.355217084288597
  Accuracy:   18.951481602084012%
--------------------
Epoch num: 60
  Train Loss: 1.6304567167535424
  Test Loss:  2.185480604569117
  Accuracy:   20.123738196027354%
--------------------
Epoch num: 80
  Train Loss: 1.585909450116257
  Test Loss:  2.1525136108199754
  Accuracy:   19.14685770107457%
--------------------


[I 2025-11-10 23:44:37,230] Trial 1 finished with value: 2.165491536259651 and parameters: {'batch_size': 64, 'num_layers': 6, 'hidden_size': 16, 'lr': 0.0001, 'epochs': 100, 'weight_decay': 0.001, 'kernel_size': 7}. Best is trial 1 with value: 2.165491536259651.


...

Epoch num: 0
  Train Loss: 2.0561250699684024
  Test Loss:  2.039267674088478
  Accuracy:   16.639531097362422%
--------------------
Epoch num: 0
  Train Loss: 2.0569631718099117
  Test Loss:  2.0361354847749076
  Accuracy:   17.55128622598502%
--------------------
Epoch num: 20
  Train Loss: 1.8585250210016966
  Test Loss:  2.101903587579727
  Accuracy:   18.886356235753826%
--------------------


[I 2025-11-11 06:14:01,027] Trial 38 pruned. 


Epoch num: 40
  Train Loss: 1.765129629522562
  Test Loss:  2.0848088016112647
  Accuracy:   20.416802344513187%
--------------------
Epoch num: 0
  Train Loss: 2.050003859990587
  Test Loss:  2.035214427858591
  Accuracy:   18.625854770433083%
--------------------
Epoch num: 20
  Train Loss: 1.9662385649668674
  Test Loss:  2.073511997858683
  Accuracy:   18.36535330511234%
--------------------


[I 2025-11-11 06:17:14,126] Trial 39 pruned. 


Epoch num: 40
  Train Loss: 1.893036648320655
  Test Loss:  2.0570398084819317
  Accuracy:   19.179420384239663%
--------------------


In [22]:
for key, value in study.best_params.items():
    print(f"    {key}: {value}")

NameError: name 'study' is not defined

In [212]:
10 * 400 / 600

6.666666666666667

In [213]:
#now build model with best found params
best_params = study.best_params
batch_size = best_params["batch_size"]
num_layers = best_params["num_layers"]
hidden_size = best_params["hidden_size"]
lr = best_params["lr"]
epochs = best_params["epochs"]
kernel_size = best_params["kernel_size"]
weight_decay = best_params["weight_decay"]

In [21]:
#custom dataset with both train && val
compressedtrain = []
compressedtrainlabels = []
compressedtrain.extend(trainimages_final)
compressedtrain.extend(valimages_final)
compressedtrainlabels.extend(trainlabel_final)
compressedtrainlabels.extend(vallabels_final)
compressedDataset =  MyDataset(compressedtrain, compressedtrainlabels, transform_train)



trainloader = DataLoader(compressedDataset, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CnnArchitecture(False, 8, num_layers, hidden_size, kernel_size)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
testloss = []


for i in range(150):
    model.train()
    training_loss = 0
    for x,y in trainloader:
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        training_loss+=loss.item()
        loss.backward()
        optimizer.step()


    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for x,y in valloader:
            x = x.to(device)
            y = y.to(device)
            y_pred = model(x)
            loss = criterion(y_pred, y)
            test_loss+=loss.item()
            total += y.size(0)
            probability, predicted = torch.max(y_pred.data, 1)
            correct += (predicted == y).sum().item()
            
    if i%20==0 or i==epochs-1: 
        avg_train_loss = training_loss / len(trainloader)
        avg_test_loss = test_loss / len(valloader)
        accuracy = (100 * correct) / total

        print(f"Epoch num: {i}")
        print(f"  Train Loss: {avg_train_loss}")
        print(f"  Test Loss:  {avg_test_loss}")
        print(f"  Accuracy:   {accuracy}%")
        print("--------------------")
        testloss.append(avg_test_loss)

NameError: name 'batch_size' is not defined

In [215]:
joblib.dump(model, "cnn_emotion_prediction.pkl")

['cnn_emotion_prediction.pkl']

In [216]:
device = torch.device('cuda')
model = joblib.load('cnn_emotion_prediction.pkl')
model = model.to(device)

In [ ]:
#now let us test it on testDataset, model is not performant at all, training from scratch is not worth it!

In [217]:
test_loss = 0
total = 0
correct = 0
for x,y in testloader:
    x = x.to(device)
    y = y.to(device)
    y_pred = model(x)
    loss = criterion(y_pred, y)
    test_loss+=loss.item()
    total+=y.size(0)
    _, prediction = torch.max(y_pred, 1)
    correct+=(prediction==y).sum().item()

avg_test_loss = test_loss / len(testloader)
accuracy = (100 * correct) / total

print(f"  Test Loss:  {avg_test_loss}")
print(f"  Accuracy:   {accuracy}%")
    

  Test Loss:  1.7339561184247334
  Accuracy:   27.995971802618328%


In [174]:
#testrun for prediction
img = Image.open('emotionfolder/Train/anger/image0022969.jpg')
img = transforms_test(img)
img = img.unsqueeze(0)
img = img.to(device)
model.eval()
with torch.no_grad():
    y_pred = model(img)

In [188]:

EmotionMap = {v:k for k,v in Map.items()}

In [189]:
EmotionMap

{'anger': 0,
 'contempt': 1,
 'disgust': 2,
 'fear': 3,
 'happy': 4,
 'neutral': 5,
 'sad': 6,
 'surprise': 7}

In [192]:
#need to define this into helper file later on...
def Predict(imgpath, model):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    PredictionEmotionMap =EmotionMap.copy()
    img = Image.open(imgpath)
    img = transforms_test(img)
    img = img.unsqueeze(0)
    img = img.to(device)
    model = model.to(device)
    model.eval()
    with torch.no_grad():
        y_pred = model(img)

    prediction_confidence_array = torch.nn.functional.softmax(y_pred, dim=1)[0].cpu().numpy()
    for i in range(len(prediction_confidence_array)):
        emotion = Map[i]
        PredictionEmotionMap[emotion] = prediction_confidence_array[i]

    return PredictionEmotionMap
    #how would I convert confidence to a 100%scale cause it ain't like that right now no?
    

In [193]:
Predict('emotionfolder/Train/anger/image0022969.jpg', model)

{'anger': np.float32(0.10275879),
 'contempt': np.float32(0.10070733),
 'disgust': np.float32(0.09365553),
 'fear': np.float32(0.10301596),
 'happy': np.float32(0.16819917),
 'neutral': np.float32(0.15946205),
 'sad': np.float32(0.13645828),
 'surprise': np.float32(0.13574292)}